[example-logistic-regression](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)


In [79]:
require 'nn'
require 'optim'

In [80]:
local matio = require 'matio'
data = matio.load('ex3data1.mat')

In [81]:
trainset = {}

In [82]:
trainset.data = data.X

In [83]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

In [84]:
trainset.label = torch.Tensor(data.y):resize(5000)

In [85]:
trainset

{
  data : DoubleTensor - size: 5000x400
  label : DoubleTensor - size: 5000
}


In [86]:
numOutput = torch.max(trainset.label) - torch.min(trainset.label) + 1

In [87]:
-- ignore setmetatable for now, it is a feature beyond the scope of this tutorial. It sets the index operator.
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);

function trainset:size()
    return self.data:size(1)
end

In [88]:
model = nn.Sequential()
model:add(nn.Linear(400,10))
model:add(nn.Sigmoid())
model:add(nn.LogSoftMax())

In [89]:
criterion = nn.ClassNLLCriterion()

In [92]:
trainer = nn.StochasticGradient(model, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 1e1

In [91]:
trainset

{
  data : DoubleTensor - size: 5000x400
  size : function: 0x404e9f10
  label : DoubleTensor - size: 5000
}


In [93]:
trainer:train(trainset)

# StochasticGradient: training	


# current error = 2.0291337489327	


# current error = 1.8917949620717	


# current error = 1.8563263550791	


# current error = 1.834961496077	


...

# current error = 1.806018499216	


# current error = 1.7948741206428	


# current error = 1.7850583051223	


# current error = 1.7762490646511	


# current error = 1.7682387625958	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 1.7682387625958	



In [94]:
acc = 0.0
for i = 1, trainset:size() do
    local groundtruth = trainset.label[i]
    local prediction = model:forward(trainset.data[i])
    local confidences, indices = torch.sort(prediction, true) -- true means descending order
    if groundtruth == indices[1] then
        acc = acc + 1
    end
end
print('accuracy: ' .. 100*acc/trainset:size() .. '%')

accuracy: 85.18%	
